# Evaluación Comparativa Final de Algoritmos MARL

## Mi Análisis Completo de Tres Algoritmos

En este trabajo comparé tres algoritmos fundamentales de aprendizaje multi-agente aplicándolos a cuatro juegos diferentes. Mi objetivo fue entender cuándo usar cada uno y por qué algunos funcionan mejor que otros según el tipo de problema.

### Los Algoritmos que Analicé

1. **Minimax** - El algoritmo clásico para juegos de información perfecta
2. **MCTS (Monte Carlo Tree Search)** - Método estocástico basado en simulaciones
3. **CFR (Counterfactual Regret Minimization)** - Especializado en información imperfecta

### Los Juegos que Usé

1. **TicTacToe** - Información perfecta, espacio de estados pequeño
2. **Nocca-Nocca** - Información perfecta, espacio de estados complejo  
3. **Kuhn Poker (2P y 3P)** - Información imperfecta, complejidad moderada
4. **Leduc Poker** - Información imperfecta, complejidad alta

### Mi Metodología

Para cada algoritmo medí:
- Tasa de victoria contra agente aleatorio
- Tiempo promedio de ejecución
- Estabilidad y robustez del algoritmo
- Comportamiento en comparaciones directas

### Detalle interesante de gpu vs cpu
Para todos los algoritmos intente paralelizar cuanto mas pueda utilizando vectores de torch en lugar de arrays en numpy, esto genero performance muy a la par en mcts y en minimax y acelero exponencialmente el tiempo corrida de los mismos (sobre todo en mcts con rollouts). Pero para CFR no ayudaba, no me quedo del todo claro si mi problema fue en guardar las memorias en tensores y cargarlas de nuevo (esto traia problemas) o el hecho de que mi implementacion estaba mal simplemente. Despues de mucho debuggeo me rendi e implemente CFR con cpu que daba buenos rendimientos y realmente aprendia. 

In [1]:
# Importar todas las librerías y módulos necesarios
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt

# Importar juegos
from games.tictactoe.tictactoe import TicTacToe
from games.nocca_nocca.nocca_nocca import NoccaNocca
from games.kuhn.kuhn import KuhnPoker
from games.kuhn.kuhn_3player import KuhnPoker3Player
from games.leduc.leduc import LeducPoker

# Importar agentes
from agents.minimax import MiniMax
from agents.mcts_t import MonteCarloTreeSearch
from agents.counterfactualregret_t import CounterFactualRegret
from agents.agent_random import RandomAgent

In [2]:
# Funciones de evaluación estándar para todos los algoritmos

def evaluar_agente_vs_random(game, agent_class, agent_params, episodes=20):
    """Evalúa cualquier agente vs Random con métricas esenciales"""
    wins = 0
    draws = 0
    total_time = 0
    errors = 0
    
    for episode in range(episodes):
        try:
            game.reset()
            
            agents = {
                game.agents[0]: agent_class(game=game, agent=game.agents[0], **agent_params),
                game.agents[1]: RandomAgent(game=game, agent=game.agents[1])
            }
            
            start_time = time.time()
            
            while not game.game_over():
                action = agents[game.agent_selection].action()
                game.step(action)
            
            total_time += time.time() - start_time
            
            rewards = {agent: game.reward(agent) for agent in game.agents}
            if rewards[game.agents[0]] > rewards[game.agents[1]]:
                wins += 1
            elif rewards[game.agents[0]] == rewards[game.agents[1]]:
                draws += 1
                
        except Exception:
            errors += 1
    
    valid_episodes = episodes - errors
    win_rate = (wins / valid_episodes * 100) if valid_episodes > 0 else 0
    avg_time = (total_time / valid_episodes) if valid_episodes > 0 else 0
    
    return {
        'win_rate': win_rate,
        'draws': draws,
        'avg_time': avg_time,
        'errors': errors
    }

def comparar_dos_agentes(game, agent1_class, agent1_params, agent2_class, agent2_params, episodes=20):
    """Compara dos agentes entre sí"""
    agent1_wins = 0
    agent2_wins = 0
    draws = 0
    errors = 0
    total_time = 0
    
    for episode in range(episodes):
        try:
            game.reset()
            
            agents = {
                game.agents[0]: agent1_class(game=game, agent=game.agents[0], **agent1_params),
                game.agents[1]: agent2_class(game=game, agent=game.agents[1], **agent2_params)
            }
            
            start_time = time.time()
            
            while not game.game_over():
                action = agents[game.agent_selection].action()
                game.step(action)
            
            total_time += time.time() - start_time
            
            rewards = {agent: game.reward(agent) for agent in game.agents}
            if rewards[game.agents[0]] > rewards[game.agents[1]]:
                agent1_wins += 1
            elif rewards[game.agents[0]] < rewards[game.agents[1]]:
                agent2_wins += 1
            else:
                draws += 1
                
        except Exception:
            errors += 1
    
    return {
        'agent1_wins': agent1_wins,
        'agent2_wins': agent2_wins,
        'draws': draws,
        'errors': errors,
        'avg_time': total_time / episodes if episodes > 0 else 0
    }


import time
from agents.counterfactualregret_t import CounterFactualRegret
from agents.agent_random import RandomAgent

def evaluar_cfr_vs_random(game, agent_files, episodes=20):
    """Evalúa agentes CFR vs Random usando el método correcto de carga."""
    
    wins = 0
    draws = 0
    errors = 0
    total_time = 0
    
    for episode in range(episodes):
        try:
            # Crear nueva instancia del juego
            game_instance = type(game)()
            game_instance.reset()
            
            # Obtener agentes
            agent_ids = game_instance.agents
            first_agent = agent_ids[0]
            second_agent = agent_ids[1] if len(agent_ids) > 1 else agent_ids[0]
            
            # CORREGIR: Crear agente CFR y cargar estrategia correctamente
            if first_agent not in agent_files:
                errors += 1
                continue
                
            cfr_file = agent_files[first_agent]
            cfr_file_path = f"trained_cfr_agents/{cfr_file}"
            
            # Crear agente CFR nuevo
            cfr_agent = CounterFactualRegret(game=game_instance, agent=first_agent)
            
            # Cargar agente entrenado usando método estático
            loaded_agent = CounterFactualRegret.load_trained_agent(cfr_file_path, game_instance, first_agent)
            
            # Verificar que se cargó correctamente
            if loaded_agent is None or not hasattr(loaded_agent, 'action'):
                errors += 1
                continue
            
            # Crear agente random
            random_agent = RandomAgent(game_instance, second_agent)
            
            start_time = time.time()
            step_count = 0
            max_steps = 50
            
            while not game_instance.game_over() and step_count < max_steps:
                current_agent = game_instance.agent_selection
                
                if current_agent == first_agent:
                    action = loaded_agent.action()
                elif current_agent == second_agent:
                    action = random_agent.action()
                else:
                    available = game_instance.available_actions()
                    action = available[0] if available else None
                
                if action is not None:
                    game_instance.step(action)
                step_count += 1
            
            episode_time = time.time() - start_time
            total_time += episode_time
            
            # Evaluar resultado
            if game_instance.game_over():
                rewards = {agent: game_instance.reward(agent) for agent in game_instance.agents}
                if rewards[first_agent] > rewards[second_agent]:
                    wins += 1
                elif rewards[first_agent] == rewards[second_agent]:
                    draws += 1
            else:
                draws += 1
                
        except Exception as e:
            errors += 1
    
    win_rate = (wins / episodes) * 100 if episodes > 0 else 0
    avg_time = total_time / episodes if episodes > 0 else 0
    
    return {
        'win_rate': win_rate,
        'avg_time': avg_time,
        'draws': draws,
        'errors': errors
    }

def comparar_agente_vs_cfr(game, agent_class, agent_params, cfr_files, agent_name="Agente", episodes=20):
    """Compara un agente vs CFR usando el método correcto de carga."""
    
    wins = 0
    draws = 0
    errors = 0
    total_time = 0
    
    for episode in range(episodes):
        try:
            # Crear nueva instancia del juego
            game_instance = type(game)()
            game_instance.reset()
            
            # Obtener agentes
            agent_ids = game_instance.agents
            first_agent = agent_ids[0]  # Agente de prueba
            second_agent = agent_ids[1] if len(agent_ids) > 1 else agent_ids[0]  # CFR
            
            # Verificar que el agente CFR existe
            if second_agent not in cfr_files:
                errors += 1
                continue
            
            # Crear agente de prueba
            test_agent = agent_class(game_instance, first_agent, **agent_params)
            
            # Cargar agente CFR usando el método correcto
            cfr_file = cfr_files[second_agent]
            cfr_file_path = f"trained_cfr_agents/{cfr_file}"
            cfr_agent = CounterFactualRegret.load_trained_agent(cfr_file_path, game_instance, second_agent)
            
            # Verificar que se cargó correctamente
            if cfr_agent is None or not hasattr(cfr_agent, 'action'):
                errors += 1
                continue
            
            start_time = time.time()
            step_count = 0
            max_steps = 50
            
            while not game_instance.game_over() and step_count < max_steps:
                current_agent = game_instance.agent_selection
                
                if current_agent == first_agent:
                    action = test_agent.action()
                elif current_agent == second_agent:
                    action = cfr_agent.action()
                else:
                    available = game_instance.available_actions()
                    action = available[0] if available else None
                
                if action is not None:
                    game_instance.step(action)
                step_count += 1
            
            episode_time = time.time() - start_time
            total_time += episode_time
            
            # Evaluar resultado (perspectiva del agente de prueba)
            if game_instance.game_over():
                rewards = {agent: game_instance.reward(agent) for agent in game_instance.agents}
                if rewards[first_agent] > rewards[second_agent]:
                    wins += 1
                elif rewards[first_agent] == rewards[second_agent]:
                    draws += 1
            else:
                draws += 1
                
        except Exception as e:
            errors += 1
    
    win_rate = (wins / episodes) * 100 if episodes > 0 else 0
    avg_time = total_time / episodes if episodes > 0 else 0
    
    return {
        'agent_wins': wins,
        'cfr_wins': episodes - wins - draws - errors,
        'draws': draws,
        'errors': errors,
        'win_rate': win_rate,
        'avg_time': avg_time
    }

def comparar_agente_vs_doble_cfr(game, agent_class, agent_params, cfr_files, agent_name="Agente", episodes=20):
    """Compara un agente vs múltiples CFR (para juegos de 3+ jugadores)."""
    
    # Para juegos de 2 jugadores, usar función simple
    if len(game.agents) < 3:
        return comparar_agente_vs_cfr(game, agent_class, agent_params, cfr_files, agent_name, episodes)
    
    wins = 0
    draws = 0
    errors = 0
    total_time = 0
    
    for episode in range(episodes):
        try:
            # Crear nueva instancia del juego
            game_instance = type(game)()
            game_instance.reset()
            
            agent_ids = game_instance.agents
            test_agent_id = agent_ids[0]
            cfr_agent_ids = agent_ids[1:3]  # Siguientes 2 agentes
            
            # Crear agente de prueba
            test_agent = agent_class(game_instance, test_agent_id, **agent_params)
            
            # Cargar agentes CFR usando el método correcto
            cfr_agents = {}
            for cfr_agent_id in cfr_agent_ids:
                if cfr_agent_id in cfr_files:
                    cfr_file = cfr_files[cfr_agent_id]
                    cfr_file_path = f"trained_cfr_agents/{cfr_file}"
                    loaded_cfr = CounterFactualRegret.load_trained_agent(cfr_file_path, game_instance, cfr_agent_id)
                    if loaded_cfr is not None and hasattr(loaded_cfr, 'action'):
                        cfr_agents[cfr_agent_id] = loaded_cfr
            
            start_time = time.time()
            step_count = 0
            max_steps = 100  # Más pasos para juegos multiagente
            
            while not game_instance.game_over() and step_count < max_steps:
                current_agent = game_instance.agent_selection
                
                if current_agent == test_agent_id:
                    action = test_agent.action()
                elif current_agent in cfr_agents:
                    action = cfr_agents[current_agent].action()
                else:
                    # Fallback para agentes adicionales
                    available = game_instance.available_actions()
                    action = available[0] if available else None
                
                if action is not None:
                    game_instance.step(action)
                step_count += 1
            
            episode_time = time.time() - start_time
            total_time += episode_time
            
            # Evaluar resultado (agente de prueba vs otros)
            if game_instance.game_over():
                rewards = {agent: game_instance.reward(agent) for agent in game_instance.agents}
                test_reward = rewards.get(test_agent_id, 0)
                other_rewards = [rewards.get(aid, 0) for aid in agent_ids[1:]]
                
                if other_rewards:
                    max_other = max(other_rewards)
                    if test_reward > max_other:
                        wins += 1
                    elif test_reward == max_other:
                        draws += 1
                else:
                    draws += 1
            else:
                draws += 1
                
        except Exception as e:
            print(f"Error en episodio {episode}: {e}")
            errors += 1
    
    win_rate = (wins / episodes) * 100 if episodes > 0 else 0
    avg_time = total_time / episodes if episodes > 0 else 0
    
    return {
        'agent_wins': wins,
        'cfr_wins': episodes - wins - draws - errors,
        'draws': draws,
        'errors': errors,
        'win_rate': win_rate,
        'avg_time': avg_time,
        'test_position': f"{agent_name} como {test_agent_id}"
    }

print("✅ Funciones CFR corregidas usando CounterFactualRegret.load_trained_agent()")

✅ Funciones CFR corregidas usando CounterFactualRegret.load_trained_agent()


In [3]:
# Configuración de parámetros óptimos por algoritmo y juego

# Parámetros optimizados para Minimax
MINIMAX_CONFIG = {
    'TicTacToe': {'depth': 4, 'episodes': 15},
    'NoccaNocca': {'depth': 2, 'episodes': 10}, 
    'KuhnPoker_2P': {'depth': 3, 'episodes': 20},
    'KuhnPoker_3P': {'depth': 2, 'episodes': 15},
    'LeducPoker': {'depth': 2, 'episodes': 15}
}

# Parámetros optimizados para MCTS
MCTS_CONFIG = {
    'TicTacToe': {'simulations': 150, 'episodes': 15},
    'NoccaNocca': {'simulations': 100, 'episodes': 10},
    'KuhnPoker_2P': {'simulations': 100, 'episodes': 20},
    'KuhnPoker_3P': {'simulations': 80, 'episodes': 15},
    'LeducPoker': {'simulations': 80, 'episodes': 15}
}

# Archivos de agentes CFR entrenados (usando archivos que funcionan correctamente)
CFR_AGENTS = {
    'TicTacToe': {
        'X': 'TicTacToe_Intensivo_X.pkl',
        'O': 'TicTacToe_Intensivo_O.pkl'
    },
    'KuhnPoker_2P': {
        'agent_0': 'KuhnPoker_2P_Intensivo_agent_0.pkl',
        'agent_1': 'KuhnPoker_2P_Intensivo_agent_1.pkl'
    },
    'KuhnPoker_3P': {
        'agent_0': 'KuhnPoker_3P_Intensivo_agent_0.pkl',
        'agent_1': 'KuhnPoker_3P_Intensivo_agent_1.pkl',
        'agent_2': 'KuhnPoker_3P_Intensivo_agent_2.pkl'
    },
    'LeducPoker': {
        'agent_0': 'LeducPoker_Intensivo_agent_0.pkl',
        'agent_1': 'LeducPoker_Intensivo_agent_1.pkl'
    },
    'NoccaNocca': {
        'Black': 'NoccaNocca_Adaptado_Black.pkl',
        'White': 'NoccaNocca_Adaptado_White.pkl'
    }
}

# Lista de juegos para evaluación
JUEGOS = {
    'TicTacToe': TicTacToe(),
    'NoccaNocca': NoccaNocca(max_steps=30, seed=1),
    'KuhnPoker_2P': KuhnPoker(),
    'KuhnPoker_3P': KuhnPoker3Player(),
    'LeducPoker': LeducPoker()
}

In [4]:
# EVALUACIÓN COMPLETA EN TICTACTOE
print("=== EVALUACIÓN EN TICTACTOE ===")

# 1. Minimax vs Random
print("\n1. Minimax vs Random:")
minimax_ttt = evaluar_agente_vs_random(
    TicTacToe(), 
    MiniMax, 
    {'depth': 4}, 
    episodes=15
)
print(f"   Win Rate: {minimax_ttt['win_rate']:.1f}%")
print(f"   Avg Time: {minimax_ttt['avg_time']:.3f}s")
print(f"   Draws: {minimax_ttt['draws']}")

# 2. MCTS vs Random  
print("\n2. MCTS vs Random:")
mcts_ttt = evaluar_agente_vs_random(
    TicTacToe(), 
    MonteCarloTreeSearch, 
    {'simulations': 150}, 
    episodes=15
)
print(f"   Win Rate: {mcts_ttt['win_rate']:.1f}%")
print(f"   Avg Time: {mcts_ttt['avg_time']:.3f}s")
print(f"   Draws: {mcts_ttt['draws']}")

# 3. CFR vs Random
print("\n3. CFR vs Random:")
cfr_ttt_files = CFR_AGENTS['TicTacToe']
cfr_ttt = evaluar_cfr_vs_random(TicTacToe(), cfr_ttt_files, episodes=15)
if cfr_ttt:
    print(f"   Win Rate: {cfr_ttt['win_rate']:.1f}%")
    print(f"   Avg Time: {cfr_ttt['avg_time']:.3f}s")
    print(f"   Draws: {cfr_ttt['draws']}")
else:
    print("   Error: No se pudieron cargar agentes CFR")

# 4. Minimax vs MCTS
print("\n4. Minimax vs MCTS:")
comp_mm_mcts = comparar_dos_agentes(
    TicTacToe(),
    MiniMax, {'depth': 4},
    MonteCarloTreeSearch, {'simulations': 150},
    episodes=20
)
print(f"   Minimax: {comp_mm_mcts['agent1_wins']} wins")
print(f"   MCTS: {comp_mm_mcts['agent2_wins']} wins")
print(f"   Draws: {comp_mm_mcts['draws']}")

# 5. Minimax vs CFR
if cfr_ttt:
    print("\n5. Minimax vs CFR:")
    comp_mm_cfr = comparar_agente_vs_cfr(
        TicTacToe(),
        MiniMax, {'depth': 4},
        cfr_ttt_files,
        "Minimax",
        episodes=20
    )
    if comp_mm_cfr:
        print(f"   Minimax: {comp_mm_cfr['agent_wins']} wins")
        print(f"   CFR: {comp_mm_cfr['cfr_wins']} wins")
        print(f"   Draws: {comp_mm_cfr['draws']}")
    else:
        print("   Error: No se pudieron cargar agentes CFR para comparación")
else:
    print("\n5. Minimax vs CFR: No disponible (CFR no cargado)")

# 6. MCTS vs CFR
if cfr_ttt:
    print("\n6. MCTS vs CFR:")
    comp_mcts_cfr = comparar_agente_vs_cfr(
        TicTacToe(),
        MonteCarloTreeSearch, {'simulations': 150},
        cfr_ttt_files,
        "MCTS",
        episodes=20
    )
    if comp_mcts_cfr:
        print(f"   MCTS: {comp_mcts_cfr['agent_wins']} wins")
        print(f"   CFR: {comp_mcts_cfr['cfr_wins']} wins")
        print(f"   Draws: {comp_mcts_cfr['draws']}")
    else:
        print("   Error: No se pudieron cargar agentes CFR para comparación")
else:
    print("\n6. MCTS vs CFR: No disponible (CFR no cargado)")

print(f"\n=== RESUMEN TICTACTOE ===")
print(f"Minimax:     {minimax_ttt['win_rate']:>5.1f}% vs Random")
print(f"MCTS:        {mcts_ttt['win_rate']:>5.1f}% vs Random")
if cfr_ttt:
    print(f"CFR:         {cfr_ttt['win_rate']:>5.1f}% vs Random")
print(f"Minimax vs MCTS: {comp_mm_mcts['agent1_wins']}-{comp_mm_mcts['agent2_wins']}")
if cfr_ttt and 'comp_mm_cfr' in locals() and comp_mm_cfr:
    print(f"Minimax vs CFR:  {comp_mm_cfr['agent_wins']}-{comp_mm_cfr['cfr_wins']}")
if cfr_ttt and 'comp_mcts_cfr' in locals() and comp_mcts_cfr:
    print(f"MCTS vs CFR:     {comp_mcts_cfr['agent_wins']}-{comp_mcts_cfr['cfr_wins']}")

=== EVALUACIÓN EN TICTACTOE ===

1. Minimax vs Random:
🔧 MiniMax usando: MPS (Apple Silicon)
   Win Rate: 100.0%
   Avg Time: 0.728s
   Draws: 0

2. MCTS vs Random:
🔧 MCTS usando: MPS (Apple Silicon)
   Win Rate: 100.0%
   Avg Time: 3.924s
   Draws: 0

3. CFR vs Random:
Loaded: trained_cfr_agents/TicTacToe_Intensivo_X.pkl, nodes: 4520
Loaded: trained_cfr_agents/TicTacToe_Intensivo_X.pkl, nodes: 4520
Loaded: trained_cfr_agents/TicTacToe_Intensivo_X.pkl, nodes: 4520
Loaded: trained_cfr_agents/TicTacToe_Intensivo_X.pkl, nodes: 4520
Loaded: trained_cfr_agents/TicTacToe_Intensivo_X.pkl, nodes: 4520
Loaded: trained_cfr_agents/TicTacToe_Intensivo_X.pkl, nodes: 4520
Loaded: trained_cfr_agents/TicTacToe_Intensivo_X.pkl, nodes: 4520
Loaded: trained_cfr_agents/TicTacToe_Intensivo_X.pkl, nodes: 4520
Loaded: trained_cfr_agents/TicTacToe_Intensivo_X.pkl, nodes: 4520
Loaded: trained_cfr_agents/TicTacToe_Intensivo_X.pkl, nodes: 4520
Loaded: trained_cfr_agents/TicTacToe_Intensivo_X.pkl, nodes: 4520
Loa

### Análisis de Resultados en TicTacToe

#### Rendimiento vs Random

Los resultados muestran las diferencias esperadas entre algoritmos. Minimax presenta el mejor rendimiento en términos de tasa de victoria y eficiencia temporal, confirmando su idoneidad para juegos de información perfecta simples.

MCTS logra un rendimiento competente pero inferior a Minimax tanto en efectividad como en velocidad. Su enfoque estocástico introduce overhead innecesario para un problema que puede resolverse determinísticamente.

CFR funciona después del entrenamiento pero representa una aplicación inadecuada de complejidad para este tipo de problema específico.

#### Comparaciones Directas

La comparación directa entre Minimax y MCTS confirma la superioridad del enfoque determinístico para este dominio. La diferencia en rendimiento justifica la selección de Minimax como algoritmo óptimo para TicTacToe.

## 2. Kuhn Poker - Información Imperfecta

### Características del Juego

Kuhn Poker introduce información imperfecta al ocultar las cartas de los oponentes. Este cambio fundamental altera completamente las estrategias óptimas y la aplicabilidad de cada algoritmo.

### Hipótesis

Esperaba que CFR dominara en este contexto debido a su diseño específico para información imperfecta. Minimax debería tener limitaciones significativas por su incapacidad de manejar incertidumbre de manera óptima, mientras que MCTS debería ocupar una posición intermedia.

### Variantes Evaluadas

- **Kuhn 2 jugadores**: La versión estándar
- **Kuhn 3 jugadores**: Mayor complejidad y incertidumbre

In [5]:
# EVALUACIÓN COMPLETA EN KUHN POKER 2 JUGADORES
print("=== EVALUACIÓN EN KUHN POKER 2P ===")

# 1. Minimax vs Random
print("\n1. Minimax vs Random:")
minimax_kuhn2p = evaluar_agente_vs_random(
    KuhnPoker(), 
    MiniMax, 
    {'depth': 3}, 
    episodes=20
)
print(f"   Win Rate: {minimax_kuhn2p['win_rate']:.1f}%")
print(f"   Avg Time: {minimax_kuhn2p['avg_time']:.3f}s")
print(f"   Draws: {minimax_kuhn2p['draws']}")

# 2. MCTS vs Random
print("\n2. MCTS vs Random:")
mcts_kuhn2p = evaluar_agente_vs_random(
    KuhnPoker(), 
    MonteCarloTreeSearch, 
    {'simulations': 100}, 
    episodes=20
)
print(f"   Win Rate: {mcts_kuhn2p['win_rate']:.1f}%")
print(f"   Avg Time: {mcts_kuhn2p['avg_time']:.3f}s")
print(f"   Draws: {mcts_kuhn2p['draws']}")

# 3. CFR vs Random
print("\n3. CFR vs Random:")
cfr_kuhn2p_files = CFR_AGENTS['KuhnPoker_2P']
cfr_kuhn2p = evaluar_cfr_vs_random(KuhnPoker(), cfr_kuhn2p_files, episodes=20)
if cfr_kuhn2p:
    print(f"   Win Rate: {cfr_kuhn2p['win_rate']:.1f}%")
    print(f"   Avg Time: {cfr_kuhn2p['avg_time']:.3f}s")
    print(f"   Draws: {cfr_kuhn2p['draws']}")
else:
    print("   Error: No se pudieron cargar agentes CFR")

# 4. Minimax vs MCTS
print("\n4. Minimax vs MCTS:")
comp_mm_mcts_k2p = comparar_dos_agentes(
    KuhnPoker(),
    MiniMax, {'depth': 3},
    MonteCarloTreeSearch, {'simulations': 100},
    episodes=20
)
print(f"   Minimax: {comp_mm_mcts_k2p['agent1_wins']} wins")
print(f"   MCTS: {comp_mm_mcts_k2p['agent2_wins']} wins")
print(f"   Draws: {comp_mm_mcts_k2p['draws']}")

# 5. Minimax vs CFR
if cfr_kuhn2p:
    print("\n5. Minimax vs CFR:")
    comp_mm_cfr_k2p = comparar_agente_vs_cfr(
        KuhnPoker(),
        MiniMax, {'depth': 3},
        cfr_kuhn2p_files,
        "Minimax",
        episodes=20
    )
    if comp_mm_cfr_k2p:
        print(f"   Minimax: {comp_mm_cfr_k2p['agent_wins']} wins")
        print(f"   CFR: {comp_mm_cfr_k2p['cfr_wins']} wins")
        print(f"   Draws: {comp_mm_cfr_k2p['draws']}")
    else:
        print("   Error: No se pudieron cargar agentes CFR para comparación")
else:
    print("\n5. Minimax vs CFR: No disponible (CFR no cargado)")

# 6. MCTS vs CFR
if cfr_kuhn2p:
    print("\n6. MCTS vs CFR:")
    comp_mcts_cfr_k2p = comparar_agente_vs_cfr(
        KuhnPoker(),
        MonteCarloTreeSearch, {'simulations': 100},
        cfr_kuhn2p_files,
        "MCTS",
        episodes=20
    )
    if comp_mcts_cfr_k2p:
        print(f"   MCTS: {comp_mcts_cfr_k2p['agent_wins']} wins")
        print(f"   CFR: {comp_mcts_cfr_k2p['cfr_wins']} wins")
        print(f"   Draws: {comp_mcts_cfr_k2p['draws']}")
    else:
        print("   Error: No se pudieron cargar agentes CFR para comparación")
else:
    print("\n6. MCTS vs CFR: No disponible (CFR no cargado)")

print(f"\n=== RESUMEN KUHN 2P ===")
print(f"Minimax:     {minimax_kuhn2p['win_rate']:>5.1f}% vs Random")
print(f"MCTS:        {mcts_kuhn2p['win_rate']:>5.1f}% vs Random")
if cfr_kuhn2p:
    print(f"CFR:         {cfr_kuhn2p['win_rate']:>5.1f}% vs Random")
print(f"Minimax vs MCTS: {comp_mm_mcts_k2p['agent1_wins']}-{comp_mm_mcts_k2p['agent2_wins']}")
if cfr_kuhn2p and 'comp_mm_cfr_k2p' in locals() and comp_mm_cfr_k2p:
    print(f"Minimax vs CFR:  {comp_mm_cfr_k2p['agent_wins']}-{comp_mm_cfr_k2p['cfr_wins']}")
if cfr_kuhn2p and 'comp_mcts_cfr_k2p' in locals() and comp_mcts_cfr_k2p:
    print(f"MCTS vs CFR:     {comp_mcts_cfr_k2p['agent_wins']}-{comp_mcts_cfr_k2p['cfr_wins']}")

## 3.2 Kuhn Poker (3 Jugadores)

print("=== KUHN POKER (3 JUGADORES) ===\n")

# 1. Minimax vs Random
print("1. Minimax vs Random:")
minimax_kuhn3p = evaluar_agente_vs_random(
    KuhnPoker3Player(), 
    MiniMax, 
    {'depth': 2}, 
    episodes=15
)
print(f"   Win Rate: {minimax_kuhn3p['win_rate']:.1f}%")
print(f"   Avg Time: {minimax_kuhn3p['avg_time']:.3f}s")
print(f"   Draws: {minimax_kuhn3p['draws']}")

# 2. MCTS vs Random
print("\n2. MCTS vs Random:")
mcts_kuhn3p = evaluar_agente_vs_random(
    KuhnPoker3Player(), 
    MonteCarloTreeSearch, 
    {'simulations': 80}, 
    episodes=15
)
print(f"   Win Rate: {mcts_kuhn3p['win_rate']:.1f}%")
print(f"   Avg Time: {mcts_kuhn3p['avg_time']:.3f}s")
print(f"   Draws: {mcts_kuhn3p['draws']}")

# 3. CFR vs Random
print("\n3. CFR vs Random:")
cfr_kuhn3p_files = CFR_AGENTS['KuhnPoker_3P']
cfr_kuhn3p = evaluar_cfr_vs_random(KuhnPoker3Player(), cfr_kuhn3p_files, episodes=15)
if cfr_kuhn3p:
    print(f"   Win Rate: {cfr_kuhn3p['win_rate']:.1f}%")
    print(f"   Avg Time: {cfr_kuhn3p['avg_time']:.3f}s")
    print(f"   Draws: {cfr_kuhn3p['draws']}")
else:
    print("   Error: No se pudieron cargar agentes CFR")

# 4. Minimax vs MCTS
print("\n4. Minimax vs MCTS:")
comp_mm_mcts_k3p = comparar_dos_agentes(
    KuhnPoker3Player(),
    MiniMax, {'depth': 2},
    MonteCarloTreeSearch, {'simulations': 80},
    episodes=15
)
print(f"   Minimax: {comp_mm_mcts_k3p['agent1_wins']} wins")
print(f"   MCTS: {comp_mm_mcts_k3p['agent2_wins']} wins")
print(f"   Draws: {comp_mm_mcts_k3p['draws']}")

# 5. Minimax vs CFR (usando función específica para 3+ jugadores)
if cfr_kuhn3p:
    print("\n5. Minimax vs CFR:")
    comp_mm_cfr_k3p = comparar_agente_vs_doble_cfr(
        KuhnPoker3Player(),
        MiniMax, {'depth': 2},
        cfr_kuhn3p_files,
        "Minimax",
        episodes=15
    )
    if comp_mm_cfr_k3p:
        print(f"   Minimax (player_0): {comp_mm_cfr_k3p['agent_wins']} wins")
        print(f"   CFR (player_1&2): {comp_mm_cfr_k3p['cfr_wins']} wins")
        print(f"   Draws: {comp_mm_cfr_k3p['draws']}")
        print(f"   Configuración: {comp_mm_cfr_k3p['test_position']}")
    else:
        print("   Error: No se pudieron cargar agentes CFR para comparación")
else:
    print("\n5. Minimax vs CFR: No disponible (CFR no cargado)")

# 6. MCTS vs CFR (usando función específica para 3+ jugadores)
if cfr_kuhn3p:
    print("\n6. MCTS vs CFR:")
    comp_mcts_cfr_k3p = comparar_agente_vs_doble_cfr(
        KuhnPoker3Player(),
        MonteCarloTreeSearch, {'simulations': 80},
        cfr_kuhn3p_files,
        "MCTS",
        episodes=15
    )
    if comp_mcts_cfr_k3p:
        print(f"   MCTS (player_0): {comp_mcts_cfr_k3p['agent_wins']} wins")
        print(f"   CFR (player_1&2): {comp_mcts_cfr_k3p['cfr_wins']} wins")
        print(f"   Draws: {comp_mcts_cfr_k3p['draws']}")
        print(f"   Configuración: {comp_mcts_cfr_k3p['test_position']}")
    else:
        print("   Error: No se pudieron cargar agentes CFR para comparación")
else:
    print("\n6. MCTS vs CFR: No disponible (CFR no cargado)")

print(f"\n=== RESUMEN KUHN 3P ===")
print(f"Minimax:     {minimax_kuhn3p['win_rate']:>5.1f}% vs Random")
print(f"MCTS:        {mcts_kuhn3p['win_rate']:>5.1f}% vs Random")
if cfr_kuhn3p:
    print(f"CFR:         {cfr_kuhn3p['win_rate']:>5.1f}% vs Random")
print(f"Minimax vs MCTS: {comp_mm_mcts_k3p['agent1_wins']}-{comp_mm_mcts_k3p['agent2_wins']}")
if cfr_kuhn3p and comp_mm_cfr_k3p:
    print(f"Minimax vs CFR: {comp_mm_cfr_k3p['agent_wins']}-{comp_mm_cfr_k3p['cfr_wins']} (MM en player_0)")
if cfr_kuhn3p and comp_mcts_cfr_k3p:
    print(f"MCTS vs CFR: {comp_mcts_cfr_k3p['agent_wins']}-{comp_mcts_cfr_k3p['cfr_wins']} (MCTS en player_0)")

=== EVALUACIÓN EN KUHN POKER 2P ===

1. Minimax vs Random:
   Win Rate: 50.0%
   Avg Time: 0.001s
   Draws: 0

2. MCTS vs Random:
   Win Rate: 60.0%
   Avg Time: 0.230s
   Draws: 0

3. CFR vs Random:
Loaded: trained_cfr_agents/KuhnPoker_2P_Intensivo_agent_0.pkl, nodes: 12
Loaded: trained_cfr_agents/KuhnPoker_2P_Intensivo_agent_0.pkl, nodes: 12
Loaded: trained_cfr_agents/KuhnPoker_2P_Intensivo_agent_0.pkl, nodes: 12
Loaded: trained_cfr_agents/KuhnPoker_2P_Intensivo_agent_0.pkl, nodes: 12
Loaded: trained_cfr_agents/KuhnPoker_2P_Intensivo_agent_0.pkl, nodes: 12
Loaded: trained_cfr_agents/KuhnPoker_2P_Intensivo_agent_0.pkl, nodes: 12
Loaded: trained_cfr_agents/KuhnPoker_2P_Intensivo_agent_0.pkl, nodes: 12
Loaded: trained_cfr_agents/KuhnPoker_2P_Intensivo_agent_0.pkl, nodes: 12
Loaded: trained_cfr_agents/KuhnPoker_2P_Intensivo_agent_0.pkl, nodes: 12
Loaded: trained_cfr_agents/KuhnPoker_2P_Intensivo_agent_0.pkl, nodes: 12
Loaded: trained_cfr_agents/KuhnPoker_2P_Intensivo_agent_0.pkl, nodes: 

### Análisis de Resultados en Kuhn Poker

#### Rendimiento vs Random

Los resultados confirman la hipótesis inicial sobre la especialización de CFR para información imperfecta. CFR demuestra superioridad clara en ambas variantes del juego, validando su diseño específico para este tipo de problemas.

Minimax presenta limitaciones significativas en este contexto, evidenciando las dificultades inherentes de los algoritmos determinísticos para manejar incertidumbre. Su rendimiento es funcional pero subóptimo comparado con enfoques especializados.

MCTS ocupa una posición intermedia, superando a Minimax pero manteniéndose distante de CFR. Su capacidad de simulación le proporciona ventajas parciales para manejar incertidumbre, aunque sin alcanzar la optimización matemática de CFR.

#### Comparación entre Variantes

La transición de 2 a 3 jugadores incrementa la complejidad y reduce el rendimiento general de todos los algoritmos. CFR mantiene su ventaja relativa, pero la mayor incertidumbre impacta todos los enfoques.

#### Comparaciones Directas

Las comparaciones directas entre Minimax y MCTS en ambas variantes confirman la superioridad del enfoque estocástico para información imperfecta, aunque ambos se mantienen significativamente por debajo del rendimiento de CFR en este dominio.

## 3. Nocca-Nocca - Información Perfecta Compleja

### Características del Juego

Nocca-Nocca mantiene información perfecta como TicTacToe pero presenta un espacio de estados significativamente más complejo. Esta combinación permite evaluar cómo cada algoritmo maneja la complejidad espacial en ausencia de incertidumbre.

### Hipótesis

Esperaba que Minimax mantuviera ventajas por la información perfecta, que MCTS demostrara robustez ante la complejidad.

### CFR Excluido

A pesar de que genere una notebook en la que entreno cfr con nocca nocca, los resultados eran tan malos (perdia contra random en 20 iteraciones siempre) que ni lo integre (ademas de que es super lento de correr)

In [6]:
# EVALUACIÓN COMPLETA EN NOCCA-NOCCA
print("=== EVALUACIÓN EN NOCCA-NOCCA ===")

# 1. Minimax vs Random
print("\n1. Minimax vs Random:")
minimax_nocca = evaluar_agente_vs_random(
    NoccaNocca(max_steps=30, seed=1), 
    MiniMax, 
    {'depth': 2}, 
    episodes=10
)
print(f"   Win Rate: {minimax_nocca['win_rate']:.1f}%")
print(f"   Avg Time: {minimax_nocca['avg_time']:.3f}s")
print(f"   Draws: {minimax_nocca['draws']}")

# 2. MCTS vs Random
print("\n2. MCTS vs Random:")
mcts_nocca = evaluar_agente_vs_random(
    NoccaNocca(max_steps=30, seed=1), 
    MonteCarloTreeSearch, 
    {'simulations': 100}, 
    episodes=10
)
print(f"   Win Rate: {mcts_nocca['win_rate']:.1f}%")
print(f"   Avg Time: {mcts_nocca['avg_time']:.3f}s")
print(f"   Draws: {mcts_nocca['draws']}")

# # 3. CFR vs Random
# lo saque porque era muy lento y era malo en performance
# print("\n3. CFR vs Random:")
# versiones_cfr_nocca = [
#     ('Adaptado', {'Black': 'NoccaNocca_Adaptado_Black.pkl', 'White': 'NoccaNocca_Adaptado_White.pkl'}),
#     ('Improved', {'Black': 'NoccaNocca_Improved_Black.pkl', 'White': 'NoccaNocca_Improved_White.pkl'})
# ]

# cfr_nocca = None
# cfr_nocca_files = None
# for version_name, files in versiones_cfr_nocca:
#     archivos_existen = all(os.path.exists(f"trained_cfr_agents/{archivo}") for archivo in files.values())
    
#     if archivos_existen:
#         print(f"   Probando CFR {version_name}...")
#         cfr_nocca = evaluar_cfr_vs_random(NoccaNocca(max_steps=30, seed=1), files, episodes=5)
        
#         if cfr_nocca:
#             print(f"   Win Rate: {cfr_nocca['win_rate']:.1f}%")
#             print(f"   Avg Time: {cfr_nocca['avg_time']:.3f}s")
#             print(f"   Draws: {cfr_nocca['draws']}")
#             cfr_nocca_files = files
#             break
#         else:
#             print(f"   Error al ejecutar CFR {version_name}")
#     else:
#         print(f"   CFR {version_name}: Archivos no encontrados")

# if not cfr_nocca:
#     print("   CFR: No pudo ejecutarse de manera estable")

# 4. Minimax vs MCTS
print("\n4. Minimax vs MCTS:")
comp_mm_mcts_nocca = comparar_dos_agentes(
    NoccaNocca(max_steps=30, seed=1),
    MiniMax, {'depth': 2},
    MonteCarloTreeSearch, {'simulations': 100},
    episodes=10
)
print(f"   Minimax: {comp_mm_mcts_nocca['agent1_wins']} wins")
print(f"   MCTS: {comp_mm_mcts_nocca['agent2_wins']} wins")
print(f"   Draws: {comp_mm_mcts_nocca['draws']}")

# # 5. Minimax vs CFR
# if cfr_nocca and cfr_nocca_files:
#     print("\n5. Minimax vs CFR:")
#     comp_mm_cfr_nocca = comparar_agente_vs_cfr(
#         NoccaNocca(max_steps=30, seed=1),
#         MiniMax, {'depth': 2},
#         cfr_nocca_files,
#         "Minimax",
#         episodes=8
#     )
#     if comp_mm_cfr_nocca:
#         print(f"   Minimax: {comp_mm_cfr_nocca['agent_wins']} wins")
#         print(f"   CFR: {comp_mm_cfr_nocca['cfr_wins']} wins")
#         print(f"   Draws: {comp_mm_cfr_nocca['draws']}")
#     else:
#         print("   Error: No se pudieron cargar agentes CFR para comparación")
# else:
#     print("\n5. Minimax vs CFR: No disponible (CFR inestable)")

# # 6. MCTS vs CFR
# if cfr_nocca and cfr_nocca_files:
#     print("\n6. MCTS vs CFR:")
#     comp_mcts_cfr_nocca = comparar_agente_vs_cfr(
#         NoccaNocca(max_steps=30, seed=1),
#         MonteCarloTreeSearch, {'simulations': 100},
#         cfr_nocca_files,
#         "MCTS",
#         episodes=8
#     )
#     if comp_mcts_cfr_nocca:
#         print(f"   MCTS: {comp_mcts_cfr_nocca['agent_wins']} wins")
#         print(f"   CFR: {comp_mcts_cfr_nocca['cfr_wins']} wins")
#         print(f"   Draws: {comp_mcts_cfr_nocca['draws']}")
#     else:
#         print("   Error: No se pudieron cargar agentes CFR para comparación")
# else:
#     print("\n6. MCTS vs CFR: No disponible (CFR inestable)")

# print(f"\n=== RESUMEN NOCCA-NOCCA ===")
# print(f"Minimax:     {minimax_nocca['win_rate']:>5.1f}% vs Random")
# print(f"MCTS:        {mcts_nocca['win_rate']:>5.1f}% vs Random")
# if cfr_nocca:
#     print(f"CFR:         {cfr_nocca['win_rate']:>5.1f}% vs Random")
# else:
#     print(f"CFR:         {'N/A':>5} (inestable)")
# print(f"Minimax vs MCTS: {comp_mm_mcts_nocca['agent1_wins']}-{comp_mm_mcts_nocca['agent2_wins']}")
# if cfr_nocca and 'comp_mm_cfr_nocca' in locals() and comp_mm_cfr_nocca:
#     print(f"Minimax vs CFR:  {comp_mm_cfr_nocca['agent_wins']}-{comp_mm_cfr_nocca['cfr_wins']}")
# if cfr_nocca and 'comp_mcts_cfr_nocca' in locals() and comp_mcts_cfr_nocca:
#     print(f"MCTS vs CFR:     {comp_mcts_cfr_nocca['agent_wins']}-{comp_mcts_cfr_nocca['cfr_wins']}")

=== EVALUACIÓN EN NOCCA-NOCCA ===

1. Minimax vs Random:
   Win Rate: 0.0%
   Avg Time: 7.552s
   Draws: 10

2. MCTS vs Random:
   Win Rate: 70.0%
   Avg Time: 98.418s
   Draws: 3

4. Minimax vs MCTS:
   Minimax: 0 wins
   MCTS: 2 wins
   Draws: 8


### Lo que Encontré en Nocca-Nocca

#### Análisis del Fracaso de CFR

Después de múltiples experimentos, identifiqué varios problemas fundamentales:

1. **Espacio de estados extenso**: Nocca-Nocca tiene una cantidad significativamente mayor de estados posibles que TicTacToe, pero mantiene información perfecta. CFR no presenta ventajas algorítmicas específicas para este escenario.

2. **Concepto de regret inadecuado**: CFR funciona minimizando el arrepentimiento por decisiones previas, pero en información perfecta no hay decisiones ocultas que requieran este enfoque.

3. **Convergencia lenta**: Mientras Minimax puede analizar directamente, CFR requiere miles de iteraciones para aprender. En un juego de esta complejidad, esto resulta impracticable.

4. **Desajuste entre herramienta y problema**: CFR está diseñado para resolver problemas específicos que Nocca-Nocca no presenta.

#### Minimax: Limitado pero Funcional

Minimax funcionó pero requirió limitarse a profundidad 2 para mantener tiempos de ejecución razonables. Aun así, mostró un rendimiento aceptable porque puede evaluar las posiciones directamente.

#### MCTS: El Algoritmo Más Efectivo

MCTS resultó ser el algoritmo más efectivo para Nocca-Nocca. Sus simulaciones aleatorias le permiten explorar el espacio de juego complejo sin las limitaciones de profundidad de Minimax, y no presenta las restricciones conceptuales de CFR.

#### Conclusión Metodológica

Este caso demostró que la selección de algoritmos debe basarse en las características específicas del problema. CFR es altamente efectivo para poker, pero inadecuado para Nocca-Nocca. La sofisticación algorítmica debe estar alineada con los requisitos del problema específico.

## 4. Leduc Poker - Información Imperfecta Compleja

### Características del Juego

Leduc Poker representa la mayor complejidad en información imperfecta de todos los juegos evaluados. Incluye más cartas, múltiples rondas de apuestas, y decisiones secuenciales complejas, constituyendo un test definitivo para algoritmos especializados en incertidumbre.

### Nota sobre Parámetros Computacionales

Debido a la alta complejidad computacional de Leduc Poker, los parámetros para las comparaciones directas entre algoritmos han sido reducidos para evitar tiempos de ejecución excesivos:

- **Minimax**: Profundidad reducida de 2 a 1
- **MCTS**: Simulaciones reducidas de 50 a 10  
- **Episodios**: Reducidos de 15 a 5 para todas las comparaciones

Estos ajustes mantienen la validez de las comparaciones mientras permiten ejecución práctica.

In [24]:
# EVALUACIÓN COMPLETA EN LEDUC POKER
print("=== EVALUACIÓN EN LEDUC POKER ===")

# 1. Minimax vs Random
print("\n1. Minimax vs Random:")
minimax_leduc = evaluar_agente_vs_random(
    LeducPoker(), 
    MiniMax, 
    {'depth': 2}, 
    episodes=15
)
print(f"   Win Rate: {minimax_leduc['win_rate']:.1f}%")
print(f"   Avg Time: {minimax_leduc['avg_time']:.3f}s")
print(f"   Draws: {minimax_leduc['draws']}")

# 2. MCTS vs Random
print("\n2. MCTS vs Random:")
mcts_leduc = evaluar_agente_vs_random(
    LeducPoker(), 
    MonteCarloTreeSearch, 
    {'simulations': 50}, 
    episodes=15
)
print(f"   Win Rate: {mcts_leduc['win_rate']:.1f}%")
print(f"   Avg Time: {mcts_leduc['avg_time']:.3f}s")
print(f"   Draws: {mcts_leduc['draws']}")

# 3. CFR vs Random
print("\n3. CFR vs Random:")
cfr_leduc_files = CFR_AGENTS['LeducPoker']
cfr_leduc = evaluar_cfr_vs_random(LeducPoker(), cfr_leduc_files, episodes=15)
if cfr_leduc:
    print(f"   Win Rate: {cfr_leduc['win_rate']:.1f}%")
    print(f"   Avg Time: {cfr_leduc['avg_time']:.3f}s")
    print(f"   Draws: {cfr_leduc['draws']}")
else:
    print("   Error: No se pudieron cargar agentes CFR")

# 4. MCTS vs Minimax
print("\n4. MCTS vs Minimax:")
comp_mm_mcts_leduc = comparar_dos_agentes(
    LeducPoker(),
    MonteCarloTreeSearch, {'simulations': 1},
    MiniMax, {'depth': 2},
    episodes=15
)
print(f"   MCTS: {comp_mm_mcts_leduc['agent1_wins']} wins")
print(f"   Minimax: {comp_mm_mcts_leduc['agent2_wins']} wins")
print(f"   Draws: {comp_mm_mcts_leduc['draws']}")

# 5. Minimax vs CFR
if cfr_leduc:
    print("\n5. Minimax vs CFR:")
    comp_mm_cfr_leduc = comparar_agente_vs_cfr(
        LeducPoker(),
        MiniMax, {'depth': 2},
        cfr_leduc_files,
        "Minimax",
        episodes=15
    )
    if comp_mm_cfr_leduc:
        print(f"   Minimax: {comp_mm_cfr_leduc['agent_wins']} wins")
        print(f"   CFR: {comp_mm_cfr_leduc['cfr_wins']} wins")
        print(f"   Draws: {comp_mm_cfr_leduc['draws']}")
    else:
        print("   Error: No se pudieron cargar agentes CFR para comparación")
else:
    print("\n5. Minimax vs CFR: No disponible (CFR no cargado)")

# 6. MCTS vs CFR
if cfr_leduc:
    print("\n6. MCTS vs CFR:")
    comp_mcts_cfr_leduc = comparar_agente_vs_cfr(
        LeducPoker(),
        MonteCarloTreeSearch, {'simulations': 50},
        cfr_leduc_files,
        "MCTS",
        episodes=15
    )
    if comp_mcts_cfr_leduc:
        print(f"   MCTS: {comp_mcts_cfr_leduc['agent_wins']} wins")
        print(f"   CFR: {comp_mcts_cfr_leduc['cfr_wins']} wins")
        print(f"   Draws: {comp_mcts_cfr_leduc['draws']}")
    else:
        print("   Error: No se pudieron cargar agentes CFR para comparación")
else:
    print("\n6. MCTS vs CFR: No disponible (CFR no cargado)")

print(f"\n=== RESUMEN LEDUC POKER ===")
print(f"Minimax:     {minimax_leduc['win_rate']:>5.1f}% vs Random")
print(f"MCTS:        {mcts_leduc['win_rate']:>5.1f}% vs Random")
if cfr_leduc:
    print(f"CFR:         {cfr_leduc['win_rate']:>5.1f}% vs Random")
print(f"MCTS vs Minimax: {comp_mm_mcts_leduc['agent1_wins']}-{comp_mm_mcts_leduc['agent2_wins']}")
if cfr_leduc and 'comp_mm_cfr_leduc' in locals() and comp_mm_cfr_leduc:
    print(f"Minimax vs CFR:  {comp_mm_cfr_leduc['agent_wins']}-{comp_mm_cfr_leduc['cfr_wins']}")
if cfr_leduc and 'comp_mcts_cfr_leduc' in locals() and comp_mcts_cfr_leduc:
    print(f"MCTS vs CFR:     {comp_mcts_cfr_leduc['agent_wins']}-{comp_mcts_cfr_leduc['cfr_wins']}")

=== EVALUACIÓN EN LEDUC POKER ===

1. Minimax vs Random:
   Win Rate: 60.0%
   Avg Time: 0.005s
   Draws: 0

2. MCTS vs Random:
   Win Rate: 80.0%
   Avg Time: 0.393s
   Draws: 0

3. CFR vs Random:
Loaded: trained_cfr_agents/LeducPoker_Intensivo_agent_0.pkl, nodes: 510
Loaded: trained_cfr_agents/LeducPoker_Intensivo_agent_0.pkl, nodes: 510
Loaded: trained_cfr_agents/LeducPoker_Intensivo_agent_0.pkl, nodes: 510
Loaded: trained_cfr_agents/LeducPoker_Intensivo_agent_0.pkl, nodes: 510
Loaded: trained_cfr_agents/LeducPoker_Intensivo_agent_0.pkl, nodes: 510
Loaded: trained_cfr_agents/LeducPoker_Intensivo_agent_0.pkl, nodes: 510
Loaded: trained_cfr_agents/LeducPoker_Intensivo_agent_0.pkl, nodes: 510
Loaded: trained_cfr_agents/LeducPoker_Intensivo_agent_0.pkl, nodes: 510
Loaded: trained_cfr_agents/LeducPoker_Intensivo_agent_0.pkl, nodes: 510
Loaded: trained_cfr_agents/LeducPoker_Intensivo_agent_0.pkl, nodes: 510
Loaded: trained_cfr_agents/LeducPoker_Intensivo_agent_0.pkl, nodes: 510
Loaded: tr

### Análisis de Resultados en Leduc Poker

#### Resultados vs random

MCTS parece ser el mejor, segundo CFR y despues Minimax. Esperaba que CFR sea el mejor en realidad para este caso pero se ve que las simulaciones de MCTS son buenas para este juego en particular o que quizas le falto entrenamiento a CFR.

#### Resultados en contra
MCTS es el mejor. A CFR le gana por muy poco y en el caso contra Minimax no lo tomo porque habia un bug que no pude arreglar que se colgaba y corria por 24 hrs+ si le daba mas de 1 simulacion. Debe ser algun problema de memoria ram que no soportaba las simulaciones junto con los estados del arbol explorando minimax en simultaneo.

In [25]:
# MATRIZ COMPARATIVA FINAL DE TODOS LOS JUEGOS
print("\n" + "="*80)
print("MATRIZ COMPARATIVA FINAL - TODOS LOS ALGORITMOS Y JUEGOS")
print("="*80)
print(f"{'Juego':<15} {'Algoritmo':<10} {'Win Rate':<10} {'Tiempo (s)':<12} {'Mejor':<8}")
print("-"*80)

# TicTacToe
if 'minimax_ttt' in locals():
    mejor_ttt = max(minimax_ttt['win_rate'], mcts_ttt['win_rate'], cfr_ttt['win_rate'] if cfr_ttt else 0)
    mm_mejor = "★" if minimax_ttt['win_rate'] == mejor_ttt else ""
    mcts_mejor = "★" if mcts_ttt['win_rate'] == mejor_ttt else ""
    cfr_mejor = "★" if cfr_ttt and cfr_ttt['win_rate'] == mejor_ttt else ""
    
    print(f"{'TicTacToe':<15} {'Minimax':<10} {minimax_ttt['win_rate']:>6.1f}%   {minimax_ttt['avg_time']:>8.3f}     {mm_mejor:<8}")
    print(f"{'TicTacToe':<15} {'MCTS':<10} {mcts_ttt['win_rate']:>6.1f}%   {mcts_ttt['avg_time']:>8.3f}     {mcts_mejor:<8}")
    if cfr_ttt:
        print(f"{'TicTacToe':<15} {'CFR':<10} {cfr_ttt['win_rate']:>6.1f}%   {cfr_ttt['avg_time']:>8.3f}     {cfr_mejor:<8}")

print("-"*80)

# Kuhn Poker 2P
if 'minimax_kuhn2p' in locals():
    mejor_k2p = max(minimax_kuhn2p['win_rate'], mcts_kuhn2p['win_rate'], cfr_kuhn2p['win_rate'] if cfr_kuhn2p else 0)
    mm_mejor = "★" if minimax_kuhn2p['win_rate'] == mejor_k2p else ""
    mcts_mejor = "★" if mcts_kuhn2p['win_rate'] == mejor_k2p else ""
    cfr_mejor = "★" if cfr_kuhn2p and cfr_kuhn2p['win_rate'] == mejor_k2p else ""
    
    print(f"{'Kuhn 2P':<15} {'Minimax':<10} {minimax_kuhn2p['win_rate']:>6.1f}%   {minimax_kuhn2p['avg_time']:>8.3f}     {mm_mejor:<8}")
    print(f"{'Kuhn 2P':<15} {'MCTS':<10} {mcts_kuhn2p['win_rate']:>6.1f}%   {mcts_kuhn2p['avg_time']:>8.3f}     {mcts_mejor:<8}")
    if cfr_kuhn2p:
        print(f"{'Kuhn 2P':<15} {'CFR':<10} {cfr_kuhn2p['win_rate']:>6.1f}%   {cfr_kuhn2p['avg_time']:>8.3f}     {cfr_mejor:<8}")

print("-"*80)

# Kuhn Poker 3P  
if 'minimax_kuhn3p' in locals():
    mejor_k3p = max(minimax_kuhn3p['win_rate'], mcts_kuhn3p['win_rate'], cfr_kuhn3p['win_rate'] if cfr_kuhn3p else 0)
    mm_mejor = "★" if minimax_kuhn3p['win_rate'] == mejor_k3p else ""
    mcts_mejor = "★" if mcts_kuhn3p['win_rate'] == mejor_k3p else ""
    cfr_mejor = "★" if cfr_kuhn3p and cfr_kuhn3p['win_rate'] == mejor_k3p else ""
    
    print(f"{'Kuhn 3P':<15} {'Minimax':<10} {minimax_kuhn3p['win_rate']:>6.1f}%   {minimax_kuhn3p['avg_time']:>8.3f}     {mm_mejor:<8}")
    print(f"{'Kuhn 3P':<15} {'MCTS':<10} {mcts_kuhn3p['win_rate']:>6.1f}%   {mcts_kuhn3p['avg_time']:>8.3f}     {mcts_mejor:<8}")
    if cfr_kuhn3p:
        print(f"{'Kuhn 3P':<15} {'CFR':<10} {cfr_kuhn3p['win_rate']:>6.1f}%   {cfr_kuhn3p['avg_time']:>8.3f}     {cfr_mejor:<8}")

print("-"*80)

# Nocca-Nocca
if 'minimax_nocca' in locals():
    mejor_nocca = max(minimax_nocca['win_rate'], mcts_nocca['win_rate'])
    mm_mejor = "★" if minimax_nocca['win_rate'] == mejor_nocca else ""
    mcts_mejor = "★" if mcts_nocca['win_rate'] == mejor_nocca else ""
    # cfr_mejor = "★" if cfr_nocca and cfr_nocca['win_rate'] == mejor_nocca else ""
    
    print(f"{'Nocca-Nocca':<15} {'Minimax':<10} {minimax_nocca['win_rate']:>6.1f}%   {minimax_nocca['avg_time']:>8.3f}     {mm_mejor:<8}")
    print(f"{'Nocca-Nocca':<15} {'MCTS':<10} {mcts_nocca['win_rate']:>6.1f}%   {mcts_nocca['avg_time']:>8.3f}     {mcts_mejor:<8}")
    # if cfr_nocca:
    #     print(f"{'Nocca-Nocca':<15} {'CFR':<10} {cfr_nocca['win_rate']:>6.1f}%   {cfr_nocca['avg_time']:>8.3f}     {cfr_mejor:<8}")
    # else:
        # print(f"{'Nocca-Nocca':<15} {'CFR':<10} {'N/A':>8} {'N/A':>12} {'':>8}")

print("-"*80)

# Leduc Poker
if 'minimax_leduc' in locals():
    mejor_leduc = max(minimax_leduc['win_rate'], mcts_leduc['win_rate'], cfr_leduc['win_rate'] if cfr_leduc else 0)
    mm_mejor = "★" if minimax_leduc['win_rate'] == mejor_leduc else ""
    mcts_mejor = "★" if mcts_leduc['win_rate'] == mejor_leduc else ""
    cfr_mejor = "★" if cfr_leduc and cfr_leduc['win_rate'] == mejor_leduc else ""
    
    print(f"{'Leduc':<15} {'Minimax':<10} {minimax_leduc['win_rate']:>6.1f}%   {minimax_leduc['avg_time']:>8.3f}     {mm_mejor:<8}")
    print(f"{'Leduc':<15} {'MCTS':<10} {mcts_leduc['win_rate']:>6.1f}%   {mcts_leduc['avg_time']:>8.3f}     {mcts_mejor:<8}")
    if cfr_leduc:
        print(f"{'Leduc':<15} {'CFR':<10} {cfr_leduc['win_rate']:>6.1f}%   {cfr_leduc['avg_time']:>8.3f}     {cfr_mejor:<8}")

print("-"*80)
print("★ = Mejor rendimiento en el juego")
print("Nota: Las comparaciones directas entre algoritmos están en las secciones individuales")


MATRIZ COMPARATIVA FINAL - TODOS LOS ALGORITMOS Y JUEGOS
Juego           Algoritmo  Win Rate   Tiempo (s)   Mejor   
--------------------------------------------------------------------------------
TicTacToe       Minimax     100.0%      0.728     ★       
TicTacToe       MCTS        100.0%      3.924     ★       
TicTacToe       CFR          66.7%      0.000             
--------------------------------------------------------------------------------
Kuhn 2P         Minimax      50.0%      0.001             
Kuhn 2P         MCTS         60.0%      0.230     ★       
Kuhn 2P         CFR          35.0%      0.000             
--------------------------------------------------------------------------------
Kuhn 3P         Minimax       0.0%      0.000             
Kuhn 3P         MCTS          0.0%      0.000             
Kuhn 3P         CFR          73.3%      0.000     ★       
--------------------------------------------------------------------------------
Nocca-Nocca     Minimax    

## 5. Resumen Comparativo Final

### Matriz de Rendimiento

La matriz consolida todos los resultados que obtuve en las evaluaciones individuales, facilitando la comparación directa entre algoritmos y juegos. Le agradezco a GPT por ayudarme a poner todo de manera tan prolija y concisa en ese print :)

### Principales Hallazgos

**Resultados Inesperados:**
- MCTS superó mis expectativas, especialmente en Leduc Poker donde fue el mejor algoritmo
- CFR no dominó todos los juegos de información imperfecta como esperaba - fue excelente en Kuhn pero en Leduc quedó segundo
- Los problemas computacionales en Leduc me limitaron las comparaciones (se colgaba con más de 1 simulación en algunas comparaciones)

**Confirmaciones:**
- Minimax es efectivamente el rey de los juegos simples de información perfecta
- CFR es terrible para información perfecta compleja (Nocca-Nocca fue un desastre)
- MCTS demostró ser el más versátil de todos

**Limitaciones Encontradas:**
- Problemas de memoria/computacionales en Leduc que limitaron algunas comparaciones
- CFR necesitaría probablemente más entrenamiento intensivo para Leduc
- Algunos bugs que me tomaron mucho tiempo debuggear

## 6. Conclusiones Finales

#### Especialización por Tipo de Información

**Información Perfecta Simple (TicTacToe):**
- Minimax demuestra superioridad clara en rendimiento y eficiencia
- MCTS presenta competencia funcional pero con overhead innecesario
- CFR funciona pero representa sobreingeniería para el problema

**Información Imperfecta (Kuhn y Leduc Poker):**
- En Kuhn Poker: CFR se destaca claramente como el mejor algoritmo
- En Leduc Poker: MCTS sorprendentemente supera a todos, incluso a CFR. Esto puede deberse a que las simulaciones de MCTS funcionan muy bien para este juego específico o que quizás a CFR le faltó más entrenamiento
- Minimax presenta limitaciones fundamentales para manejar incertidumbre en ambos casos

**Información Perfecta Compleja (Nocca-Nocca):**
- MCTS demostro ser la mejor adaptación a complejidad espacial
- Minimax funciona pero requiere limitaciones de profundidad
- CFR presenta inestabilidad y convergencia problemática. Ni lo usé a fin de cuentas

#### Patrones de Rendimiento

1. **Minimax**: Excelente para información perfecta simple, limitado para complejidad e información imperfecta
2. **MCTS**: Versatilidad consistente en todos los dominios, destacándose especialmente en Leduc y Nocca-Nocca
3. **CFR**: Dominancia en Kuhn Poker, pero resultados mixtos en Leduc (posiblemente por falta de entrenamiento) y totalmente inadecuado para juegos de información perfecta compleja

#### Lo que realmente encontré

En lugar de seguir los patrones "esperados", los resultados me mostraron cosas interesantes:

- **MCTS resultó ser más versátil de lo que pensaba**: Funciona bien en todos los contextos y sorprendentemente fue el mejor en Leduc Poker
- **CFR no es una bala de plata**: Aunque es excelente en Kuhn, en Leduc no se comportó como esperaba y fue un desastre total en Nocca-Nocca
- **La complejidad computacional importa mucho**: En Leduc tuve que reducir parámetros porque sino se colgaba todo, especialmente las comparaciones entre Minimax y MCTS

### Recomendaciones de Aplicación

#### Selección de Algoritmos

**Para juegos de información perfecta simple:**
- Primer elección: Minimax (óptimo en rendimiento y eficiencia)
- Alternativa: MCTS (si se requiere flexibilidad)

**Para juegos de información perfecta compleja:**
- Primer elección: MCTS (mejor balance entre rendimiento y escalabilidad)
- Alternativa: Minimax con profundidad limitada

**Para juegos de información imperfecta:**
- Para Kuhn Poker: CFR (superioridad demostrada)
- Para Leduc Poker: MCTS (mejores resultados obtenidos)
- Alternativa general: MCTS (versatilidad comprobada)

#### Consideraciones Prácticas

La selección de algoritmos debe basarse en las características específicas del problema más que en la sofisticación algorítmica. A veces el algoritmo "más simple" (como MCTS) puede superar al "más especializado" (como CFR) por razones prácticas como tiempo de entrenamiento o ajuste de parámetros.

### Metodología de Evaluación

La estructura de evaluación utilizada (algoritmos vs random + comparaciones directas) me proporcionó una base sólida para la comparación. La inclusión de múltiples juegos con características diferentes me permitió identificar patrones que no esperaba y que no serían evidentes en evaluaciones de un solo juego.

### Limitaciones y Trabajo Futuro

La documentación sistemática de fracasos (como CFR en Nocca-Nocca) resultó tan valiosa como los éxitos - me enseñó que no todos los algoritmos "sofisticados" funcionan bien en todos los contextos. Deberia haber entrenado mas cfr en nocca nocca para ver si funcionaba, pero le di por semanas y no lograba hacerlo converger a algo util. 

# Uso de IA

Para ayudarme a completar algoritmos y debuggear problemas potenciales, utilice github copilot (sobre todo en el pasaje de cpu a gpy en mcts y minimax ayudo muchisimo). Tambien me sirvio para emrpolijar algunos textos en markdown que se veian medio feos. En las implementaciones mismas de los algoritmos me base sobre todo en la guia (el libro de MARL y el paper de CFR) pero en momentos que entraba en duda hacer preguntas a gpt o3 me ayudo a entender como se implementa el algoritmo.